In [1]:
import settings
import helpers
import glob
import os
import cv2  # conda install -c https://conda.anaconda.org/menpo opencv3
import scipy.misc
import dicom  # pip install pydicom
import numpy
import math
from multiprocessing import Pool

Computer:  DESKTOP-F8TV69I


d:\anaconda3\envs\tensorflow\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
def load_patient(src_dir):
    slices = [dicom.read_file(src_dir + '/' + s) for s in os.listdir(src_dir)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = numpy.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = numpy.abs(slices[0].SliceLocation - slices[1].SliceLocation)

    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

In [3]:
def get_pixels_hu(slices):
    image = numpy.stack([s.pixel_array for s in slices])
    image = image.astype(numpy.int16)
    image[image == -2000] = 0
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(numpy.float64)
            image[slice_number] = image[slice_number].astype(numpy.int16)
        image[slice_number] += numpy.int16(intercept)

    return numpy.array(image, dtype=numpy.int16)

In [4]:
def extract_dicom_images_patient(src_path):
    
    patient_id = src_path
    src_path = settings.NDSB3_RAW_SRC_DIR + src_path + "/"
    
    #创建存储路径
    dst_path = settings.NDSB3_EXTRACTED_IMAGE_DIR + patient_id + "/"
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    
    #读取数据
    slices = load_patient(src_path)
    pixels = get_pixels_hu(slices)
    image = pixels
    print("Original Shape:",image.shape)
    
    #如果图片有倾斜旋转，之后需要处理
    cos_value = (slices[0].ImageOrientationPatient[0])
    cos_degree = round(math.degrees(math.acos(cos_value)),2)
    
    #如果图片是倒叙的，需要调整顺序
    invert_order = slices[1].ImagePositionPatient[2] > slices[0].ImagePositionPatient[2]
    if not invert_order:
        image = numpy.flipud(image)

    #重采样
    pixel_spacing = slices[0].PixelSpacing
    pixel_spacing.append(slices[0].SliceThickness)
    image = helpers.rescale_patient_images(image, pixel_spacing, settings.TARGET_VOXEL_MM)
    print("Rescale Shape:",image.shape)

    for i in range(image.shape[0]):
        
        #获取一张原始图片
        org_img = image[i]
        
        #用来处理图像的倾斜旋转
        if cos_degree>0.0:
            org_img = cv_flip(org_img,org_img.shape[1],org_img.shape[0],cos_degree)
        
        #肺分割
        img, mask = helpers.get_segmented_lungs(org_img.copy())
        
        #数据标准化
        org_img = helpers.normalize_hu(org_img)
        
        dst_path_final = dst_path + "img_" + str(i).rjust(4, '0') + "_i.png"
        cv2.imwrite(dst_path_final, org_img * 255)
        cv2.imwrite(dst_path_final.replace("_i.png", "_m.png"), mask * 255)

In [5]:
def extract_dicom_images(clean_targetdir_first=False, only_patient_id=None):
    
    print("Extracting images")
    
    target_dir = settings.NDSB3_EXTRACTED_IMAGE_DIR
    
    if clean_targetdir_first is True:
        
        print("Cleaning target dir")
        
        for file_path in glob.glob(target_dir + "*.*"):
            os.remove(file_path)

    if only_patient_id is None:
        
        src_paths = os.listdir(settings.NDSB3_RAW_SRC_DIR)
        
        if False:
            pool = Pool(settings.WORKER_POOL_SIZE)
            pool.map(extract_dicom_images_patient, dirs)
        else:
            count=1
            for src_path in src_paths:
                 #计数器
                print("Counter:",count)
                extract_dicom_images_patient(src_path)
                count+=1
               
    else:
        extract_dicom_images_patient(only_patient_id)

# 函数入口

In [6]:
if __name__ == '__main__':
    extract_dicom_images(clean_targetdir_first=False, only_patient_id=None)

Extracting images
Counter: 1
Original Shape: (195, 512, 512)
Rescale Shape: (390, 355, 355)
Counter: 2
Original Shape: (265, 512, 512)
Rescale Shape: (331, 298, 298)
Counter: 3
Original Shape: (233, 512, 512)
Rescale Shape: (350, 414, 414)
Counter: 4
Original Shape: (173, 512, 512)
Rescale Shape: (346, 340, 340)
Counter: 5
Original Shape: (146, 512, 512)
Rescale Shape: (292, 370, 370)
Counter: 6
Original Shape: (171, 512, 512)
Rescale Shape: (342, 340, 340)
Counter: 7
Original Shape: (123, 512, 512)
Rescale Shape: (308, 350, 350)
Counter: 8
Original Shape: (134, 512, 512)
Rescale Shape: (335, 306, 306)
Counter: 9
Original Shape: (135, 512, 512)
Rescale Shape: (338, 345, 345)
Counter: 10
Original Shape: (191, 512, 512)
Rescale Shape: (344, 340, 340)
Counter: 11
Original Shape: (217, 512, 512)
Rescale Shape: (282, 275, 275)
Counter: 12
Original Shape: (231, 512, 512)
Rescale Shape: (289, 339, 339)
Counter: 13
Original Shape: (159, 512, 512)
Rescale Shape: (318, 340, 340)
Counter: 14
Orig

Counter: 110
Original Shape: (243, 512, 512)
Rescale Shape: (304, 360, 360)
Counter: 111
Original Shape: (172, 512, 512)
Rescale Shape: (344, 283, 283)
Counter: 112
Original Shape: (224, 512, 512)
Rescale Shape: (403, 380, 380)
Counter: 113
Original Shape: (208, 512, 512)
Rescale Shape: (374, 340, 340)
Counter: 114
Original Shape: (122, 512, 512)
Rescale Shape: (305, 350, 350)
Counter: 115
Original Shape: (126, 512, 512)
Rescale Shape: (315, 350, 350)
Counter: 116
Original Shape: (151, 512, 512)
Rescale Shape: (302, 370, 370)
Counter: 117
Original Shape: (194, 512, 512)
Rescale Shape: (349, 340, 340)
Counter: 118
Original Shape: (183, 512, 512)
Rescale Shape: (329, 340, 340)
Counter: 119
Original Shape: (146, 512, 512)
Rescale Shape: (292, 380, 380)
Counter: 120
Original Shape: (184, 512, 512)
Rescale Shape: (331, 370, 370)
Counter: 121
Original Shape: (170, 512, 512)
Rescale Shape: (340, 340, 340)
Counter: 122
Original Shape: (173, 512, 512)
Rescale Shape: (346, 290, 290)
Counter: 123

Counter: 218
Original Shape: (96, 512, 512)
Rescale Shape: (240, 310, 310)
Counter: 219
Original Shape: (131, 512, 512)
Rescale Shape: (262, 295, 295)
Counter: 220
Original Shape: (501, 512, 512)
Rescale Shape: (313, 350, 350)
Counter: 221
Original Shape: (132, 512, 512)
Rescale Shape: (330, 360, 360)
Counter: 222
Original Shape: (184, 512, 512)
Rescale Shape: (331, 280, 280)
Counter: 223
Original Shape: (158, 512, 512)
Rescale Shape: (316, 350, 350)
Counter: 224
Original Shape: (188, 512, 512)
Rescale Shape: (376, 340, 340)
Counter: 225
Original Shape: (175, 512, 512)
Rescale Shape: (350, 360, 360)
Counter: 226
Original Shape: (258, 512, 512)
Rescale Shape: (322, 382, 382)
Counter: 227
Original Shape: (205, 512, 512)
Rescale Shape: (256, 293, 293)
Counter: 228
Original Shape: (138, 512, 512)
Rescale Shape: (276, 300, 300)
Counter: 229
Original Shape: (119, 512, 512)
Rescale Shape: (298, 373, 373)
Counter: 230
Original Shape: (142, 512, 512)
Rescale Shape: (355, 290, 290)
Counter: 231


Counter: 326
Original Shape: (119, 512, 512)
Rescale Shape: (298, 330, 330)
Counter: 327
Original Shape: (241, 512, 512)
Rescale Shape: (301, 288, 288)
Counter: 328
Original Shape: (176, 512, 512)
Rescale Shape: (352, 309, 309)
Counter: 329
Original Shape: (165, 512, 512)
Rescale Shape: (330, 330, 330)
Counter: 330
Original Shape: (165, 512, 512)
Rescale Shape: (330, 300, 300)
Counter: 331
Original Shape: (154, 512, 512)
Rescale Shape: (308, 290, 290)
Counter: 332
Original Shape: (173, 512, 512)
Rescale Shape: (346, 360, 360)
Counter: 333
Original Shape: (152, 512, 512)
Rescale Shape: (304, 365, 365)
Counter: 334
Original Shape: (276, 512, 512)
Rescale Shape: (345, 380, 380)
Counter: 335
Original Shape: (175, 512, 512)
Rescale Shape: (350, 367, 367)
Counter: 336
Original Shape: (219, 512, 512)
Rescale Shape: (328, 350, 350)
Counter: 337
Original Shape: (252, 512, 512)
Rescale Shape: (315, 319, 319)
Counter: 338
Original Shape: (126, 512, 512)
Rescale Shape: (315, 290, 290)
Counter: 339

Counter: 434
Original Shape: (254, 512, 512)
Rescale Shape: (330, 351, 351)
Counter: 435
Original Shape: (165, 512, 512)
Rescale Shape: (330, 343, 343)
Counter: 436
Original Shape: (153, 512, 512)
Rescale Shape: (306, 370, 370)
Counter: 437
Original Shape: (269, 512, 512)
Rescale Shape: (336, 336, 336)
Counter: 438
Original Shape: (155, 512, 512)
Rescale Shape: (310, 348, 348)
Counter: 439
Original Shape: (205, 512, 512)
Rescale Shape: (308, 330, 330)
Counter: 440
Original Shape: (167, 512, 512)
Rescale Shape: (334, 336, 336)
Counter: 441
Original Shape: (182, 512, 512)
Rescale Shape: (273, 302, 302)
Counter: 442
Original Shape: (341, 512, 512)
Rescale Shape: (341, 302, 302)
Counter: 443
Original Shape: (134, 512, 512)
Rescale Shape: (335, 350, 350)
Counter: 444
Original Shape: (171, 512, 512)
Rescale Shape: (308, 300, 300)
Counter: 445
Original Shape: (159, 512, 512)
Rescale Shape: (318, 322, 322)
Counter: 446
Original Shape: (434, 512, 512)
Rescale Shape: (271, 320, 320)
Counter: 447

...

Counter: 1081
Original Shape: (242, 512, 512)
Rescale Shape: (315, 284, 284)
Counter: 1082
Original Shape: (223, 512, 512)
Rescale Shape: (290, 280, 280)
Counter: 1083
Original Shape: (174, 512, 512)
Rescale Shape: (313, 290, 290)
Counter: 1084
Original Shape: (121, 512, 512)
Rescale Shape: (302, 314, 314)
Counter: 1085
Original Shape: (134, 512, 512)
Rescale Shape: (335, 280, 280)
Counter: 1086
Original Shape: (245, 512, 512)
Rescale Shape: (306, 360, 360)
Counter: 1087
Original Shape: (151, 512, 512)
Rescale Shape: (302, 360, 360)
Counter: 1088
Original Shape: (181, 512, 512)
Rescale Shape: (326, 298, 298)
Counter: 1089
Original Shape: (123, 512, 512)
Rescale Shape: (308, 310, 310)
Counter: 1090
Original Shape: (151, 512, 512)
Rescale Shape: (302, 360, 360)
Counter: 1091
Original Shape: (167, 512, 512)
Rescale Shape: (301, 306, 306)
Counter: 1092
Original Shape: (226, 512, 512)
Rescale Shape: (339, 346, 346)
Counter: 1093
Original Shape: (140, 512, 512)
Rescale Shape: (280, 315, 315)

Rescale Shape: (305, 380, 380)
Counter: 1188
Original Shape: (328, 512, 512)
Rescale Shape: (328, 284, 284)
Counter: 1189
Original Shape: (167, 512, 512)
Rescale Shape: (301, 350, 350)
Counter: 1190
Original Shape: (121, 512, 512)
Rescale Shape: (302, 293, 293)
Counter: 1191
Original Shape: (208, 512, 512)
Rescale Shape: (260, 340, 340)
Counter: 1192
Original Shape: (130, 512, 512)
Rescale Shape: (325, 340, 340)
Counter: 1193
Original Shape: (133, 512, 512)
Rescale Shape: (332, 351, 351)
Counter: 1194
Original Shape: (498, 512, 512)
Rescale Shape: (311, 338, 338)
Counter: 1195
Original Shape: (172, 512, 512)
Rescale Shape: (344, 290, 290)
Counter: 1196
Original Shape: (146, 512, 512)
Rescale Shape: (292, 350, 350)
Counter: 1197
Original Shape: (174, 512, 512)
Rescale Shape: (348, 312, 312)
Counter: 1198
Original Shape: (153, 512, 512)
Rescale Shape: (306, 326, 326)
Counter: 1199
Original Shape: (128, 512, 512)
Rescale Shape: (320, 319, 319)
Counter: 1200
Original Shape: (205, 512, 512)

Counter: 1294
Original Shape: (149, 512, 512)
Rescale Shape: (298, 340, 340)
Counter: 1295
Original Shape: (459, 512, 512)
Rescale Shape: (287, 339, 339)
Counter: 1296
Original Shape: (186, 512, 512)
Rescale Shape: (279, 380, 380)
Counter: 1297
Original Shape: (255, 512, 512)
Rescale Shape: (319, 360, 360)
Counter: 1298
Original Shape: (185, 512, 512)
Rescale Shape: (370, 390, 390)
Counter: 1299
Original Shape: (113, 512, 512)
Rescale Shape: (282, 330, 330)
Counter: 1300
Original Shape: (115, 512, 512)
Rescale Shape: (288, 350, 350)
Counter: 1301
Original Shape: (168, 512, 512)
Rescale Shape: (336, 302, 302)
Counter: 1302
Original Shape: (166, 512, 512)
Rescale Shape: (332, 368, 368)
Counter: 1303
Original Shape: (169, 512, 512)
Rescale Shape: (338, 340, 340)
Counter: 1304
Original Shape: (126, 512, 512)
Rescale Shape: (315, 320, 320)
Counter: 1305
Original Shape: (165, 512, 512)
Rescale Shape: (330, 340, 340)
Counter: 1306
Original Shape: (154, 512, 512)
Rescale Shape: (308, 380, 380)

Rescale Shape: (324, 320, 320)
Counter: 1401
Original Shape: (144, 512, 512)
Rescale Shape: (259, 270, 270)
Counter: 1402
Original Shape: (165, 512, 512)
Rescale Shape: (330, 312, 312)
Counter: 1403
Original Shape: (197, 512, 512)
Rescale Shape: (355, 360, 360)
Counter: 1404
Original Shape: (167, 512, 512)
Rescale Shape: (301, 286, 286)
Counter: 1405
Original Shape: (175, 512, 512)
Rescale Shape: (350, 330, 330)
Counter: 1406
Original Shape: (124, 512, 512)
Rescale Shape: (248, 360, 360)
Counter: 1407
Original Shape: (220, 512, 512)
Rescale Shape: (275, 310, 310)
Counter: 1408
Original Shape: (130, 512, 512)
Rescale Shape: (325, 400, 400)
Counter: 1409
Original Shape: (159, 512, 512)
Rescale Shape: (318, 299, 299)
Counter: 1410
Original Shape: (139, 512, 512)
Rescale Shape: (348, 320, 320)
Counter: 1411
Original Shape: (140, 512, 512)
Rescale Shape: (280, 330, 330)
Counter: 1412
Original Shape: (120, 512, 512)
Rescale Shape: (300, 310, 310)
Counter: 1413
Original Shape: (162, 512, 512)

Counter: 1507
Original Shape: (131, 512, 512)
Rescale Shape: (328, 330, 330)
Counter: 1508
Original Shape: (206, 512, 512)
Rescale Shape: (309, 380, 380)
Counter: 1509
Original Shape: (282, 512, 512)
Rescale Shape: (352, 360, 360)
Counter: 1510
Original Shape: (209, 512, 512)
Rescale Shape: (376, 360, 360)
Counter: 1511
Original Shape: (114, 512, 512)
Rescale Shape: (285, 390, 390)
Counter: 1512
Original Shape: (132, 512, 512)
Rescale Shape: (330, 334, 334)
Counter: 1513
Original Shape: (114, 512, 512)
Rescale Shape: (285, 270, 270)
Counter: 1514
Original Shape: (153, 512, 512)
Rescale Shape: (306, 310, 310)
Counter: 1515
Original Shape: (278, 512, 512)
Rescale Shape: (348, 340, 340)
Counter: 1516
Original Shape: (240, 512, 512)
Rescale Shape: (300, 384, 384)
Counter: 1517
Original Shape: (123, 512, 512)
Rescale Shape: (308, 273, 273)
Counter: 1518
Original Shape: (199, 512, 512)
Rescale Shape: (358, 350, 350)
Counter: 1519
Original Shape: (385, 512, 512)
Rescale Shape: (385, 408, 408)